In [1]:
import pyodbc
import pandas as pd
import numpy as np
from pprint import pprint
import datetime
from configparser import ConfigParser



In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Inserting in AUDIT_INTGN 

In [4]:
# Changed Code for audit integration

conn.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
(COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), 'RISK_ORGANIZATION',GETDATE(),'ERROR','RISK_ORGANIZATION','ORGANIZATION_MASTER_CLUSTER_FINGATE_REPORT',(SELECT MAX(T.MAX_DATE) FROM (SELECT MAX(CREATE_DATE) AS MAX_DATE FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_CLUSTER_FINGATE_REPORT UNION SELECT MAX(Update_Date) AS MAX_DATE FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_CLUSTER_FINGATE_REPORT) T))""")

job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN with(nolock)",conn)
job_id = job_id.iloc[0,0]
rules_df = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_LKP_RULES] with(nolock)", conn)
conn.commit()
conn.close()


In [ ]:
%store job_id

In [7]:
rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) & (rules_df['PERSON_TYPE'] == 'Organization')]

RULE_ID     RULE_NO                                          RULE_DESC  \
100   2.3.5  O002003005  Value of credit card transaction(s) is greater...   
101   2.3.6  O002003006  Value of cash transaction(s) is greater than x...   
104  2.1.12  O002001012  Entity/its parent company/ its holding company...   
105  2.1.13  O002001013  Country of incorporation and/or operating coun...   
107  2.1.18  O002001018                       Number of brokerage accounts   
111  2.1.32  O002001032                     No of High Risk NTRs filed\r\n   
112  2.1.19  O002001019                           Number of demat accounts   
113  2.1.21  O002001021  Number of private banking/wealth management ac...   
114  2.1.25  O002001025                          No of Low Risk STRs filed   
115  2.1.23  O002001023                         No of High Risk STRs filed   
116  2.1.24  O002001024                       No of Medium Risk STRs filed   
117  2.1.26  O002001026                         No of High Risk CTRs filed   
118  2.1.29  O002001029                       No of High Risk CBWTRs filed   
119   3.1.4  O003001004                              Type of accounts held   
120   2.3.1  O002003001  Total amount of credit is greater than x% of r...   
121   2.3.2  O002003002  Total amount of debit is greater than x% of re...   
122   2.3.3  O002003003  Immovable Property value is greater than x% of...   
123   2.3.7  O002003007  Value of mutual investment(s) is greater than ...   
124   2.3.8  O002003008  Value of investment(s) in capital markets is g...   
126   3.1.1  O003001001                                     Type of Entity   
127   3.1.7  O003001007  Ageing of Reports Filed (STR, CTR, CBWTR, CCR,...   
128   3.1.9  O003001009                             Client onboarding date   
129  3.1.10  O003001010                          Date of last KYC / re-KYC   
183  2.1.34  O002001034                          No of Low Risk NTRs filed   
184  2.1.33  O002001033                       No of Medium Risk NTRs filed   
185  2.1.31  O002001031                        No of Low Risk CBWTRs filed   
186  2.1.30  O002001030                     No of Medium Risk CBWTRs filed   
187  2.1.28  O002001028                          No of Low Risk CTRs filed   
205   2.1.8  O002001008                           Number of mobile Numbers   
206   2.1.9  O002001009                                Number of email Ids   
207  2.1.10  O002001010                             Number of credit cards   
208  2.1.11  O002001011               Number of prepaid/wallets/gift cards   
214   2.1.1  O002001001                            Number of bank accounts   
215   3.1.3  O003001003                          Corporation Age Risk/ DOI   
216   3.1.2  O003001002             High Risk Business/Occupation/HSN code   
217   2.1.2  O002001002                         Accounts in too many banks   
218   2.1.4  O002001004           Accounts in high Risk domestic locations   
219   2.1.5  O002001005          Addresses in high Risk domestic locations   
220  2.1.27  O002001027                       No of Medium Risk CTRs filed   
221   2.1.7  O002001007  Number of names/aliases reported across differ...   
222   3.1.8  O003001008                      Organization risk given by RE   

      PERSON_TYPE                                          RISK_TYPE  \
100  Organization         GoS-Annual Behaviour based Risk Indicators   
101  Organization         GoS-Annual Behaviour based Risk Indicators   
104  Organization  KYC profile/ Inherent risk based High Risk Ind...   
105  Organization  KYC profile/ Inherent risk based High Risk Ind...   
107  Organization  KYC profile/ Inherent risk based High Risk Ind...   
111  Organization  KYC profile/ Inherent risk based High Risk Ind...   
112  Organization  KYC profile/ Inherent risk based High Risk Ind...   
113  Organization  KYC profile/ Inherent risk based High Risk Ind...   
114  Organization  KYC profile/ Inherent risk based High Risk Ind...   
115  Or

# Queries that populate risk_person_summary_stg

In [10]:
#Queries that populate risk_person_summary_stg

master_data_pop_org_q1 = '''INSERT INTO [Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]
(
ORGANIZATION_MASTER_ID,CREATE_DATE
)
SELECT DISTINCT ORGANIZATION_MASTER_ID,GETDATE()
FROM FINCORE.ORGANIZATION_MASTER_CLUSTER_FINGATE_REPORT PMCFR WITH (NOLOCK)
INNER JOIN FINCORE.ORGANIZATION_CLUSTER PC  WITH (NOLOCK)
ON PMCFR.ORGANIZATION_CLUSTER_ID = PC.ORGANIZATION_CLUSTER_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.fingate_EntityDetail PD
ON PC.Fingate_Entity_Id = PD.ID_  AND PD.REPORTID = PMCFR.REPORT_ID
where is_active_flag = 1

'''

# org_occ_update = '''
#    UPDATE [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]
#    SET [OCCUPATION_BUSINESS] = LOB
# from
# (
# 	   SELECT Organization_Master_Id,CDR.Industial_Activity_Code_Description as LOB
# 	   FROM FINCORE_DB.FINCORE.Organization_Cluster A INNER 
# 	   JOIN EXTL_DB.MCA.Mca_Org_Master_Data CDR
# 	   ON A.Unique_Company_Id = CDR.Unique_Company_Id
# 	   inner join FINCORE_DB.FINCORE.Organization_Master_Cluster_Fingate_Report C
# 	   on C.Organization_Cluster_Id  = A.Organization_Cluster_Id) B

# where [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG].ORGANIZATION_MASTER_ID = B.Organization_Master_Id'''




org_occ_update = '''
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG 
SET OCCUPATION_BUSINESS  = OCC FROM (
SELECT * FROM (
SELECT ORGANIZATION_MASTER_ID,LOB,
CASE  WHEN LOB LIKE '%WHOLESALE%' THEN 'Wholesale'
	 WHEN (LOB LIKE '%MANUFACTU%'  OR LOB LIKE '%PRODUCTION%' )THEN 'Manufacturing'
	 WHEN (LOB LIKE '%REAL ESTATE%' OR LOB LIKE '%BUILDING%' OR LOB LIKE '%CONSTRUC%'  ) THEN 'Real_estate'
	 WHEN (LOB LIKE '%RENT%' )THEN 'Rent'
	 WHEN (LOB LIKE '%CONSULT%' )THEN 'Consultancy'
	 WHEN (LOB LIKE '%MINING%' OR LOB LIKE '%EXTRACTION%' ) THEN 'Mining'
	 WHEN (LOB LIKE '%FINANC%' OR LOB LIKE '%INSURANCE%' OR LOB LIKE '%FUND%' OR LOB LIKE '%INVESTMENT%' ) THEN 'Finance'
	 WHEN LOB LIKE '%RETAIL%' THEN 'Retail'
	 WHEN (LOB LIKE '%EDUCAT%'  OR LOB LIKE '%SCHOOL%' OR LOB LIKE '% TUTION %' OR LOB LIKE '%COACHING%') THEN 'Education' 
	 WHEN (LOB LIKE '%NURSING%'  OR LOB LIKE '%PATHOLO%' OR LOB LIKE '%DENTAL%' OR LOB LIKE '%AYURVED%' OR LOB LIKE '%HOSPITAL%'  OR LOB LIKE '%HEALTH%' OR LOB LIKE '%DIAGNO%' OR LOB LIKE '%MEDI%'  OR LOB LIKE '%HOMEOPAT%') THEN 'Healthcare'
	 WHEN (LOB LIKE '%TRAVEL%' OR LOB LIKE '%HOTEL%' OR LOB LIKE '%TOUR%' OR LOB LIKE '%GUEST HOUS%' OR LOB LIKE '%BAR%' OR LOB LIKE '%RESTAURANT%') THEN 'Tourism'
	 WHEN (LOB LIKE '%SILK%' OR LOB LIKE '%WOOL%') THEN 'Clothing'
	 WHEN (LOB LIKE '%AGRICULT%' OR LOB LIKE '%EGG%' OR LOB LIKE '%MILK%' OR LOB LIKE '%ANIMAL HUSBA%' OR LOB LIKE '%FARM%' OR LOB LIKE '%FISH%' OR LOB LIKE '%FOREST%' OR LOB LIKE '%HORTICUL%'  ) THEN 'Farm'
	 WHEN (LOB LIKE '%LOAN%' ) THEN 'Business_loan'
	 WHEN (LOB LIKE '%CASINO%' ) THEN 'Casino'
	 WHEN (LOB LIKE '%SOFTWARE%'  OR LOB LIKE '% IT %' ) THEN 'Software'
	 WHEN (LOB LIKE '%TRANSPORT%' OR LOB LIKE '%AMBULANCE%' OR LOB LIKE '%PACKERS AND MOVERS%' OR LOB LIKE '%CARGO%' OR LOB LIKE '%FREIGHT%' ) THEN 'Transport'
	 WHEN (LOB LIKE '%SPORT%'   ) THEN 'Sports'
	 WHEN (LOB LIKE '%MILL%'   ) THEN 'Mills'
	 WHEN (LOB LIKE '%TELECOM%'  OR LOB LIKE '%BROADCAST%' ) THEN 'Telecom'
	 WHEN (LOB LIKE '%SECURITY%' ) THEN 'Security'
	 WHEN (LOB LIKE '%SOCIAL%' ) THEN 'Social'
	 WHEN (LOB LIKE '%SERVICES%' ) THEN 'Services'
	 ELSE 'Others' END AS OCC
FROM (
SELECT DISTINCT C.ORGANIZATION_MASTER_ID,D.NOB_DESCRIPTION AS LOB
FROM FINCORE_DB.FINCORE.ORGANIZATION_CLUSTER A 
INNER JOIN EXTL_DB.CBDT.CBDT_ITR_DATA B ON A.PAN = B.PAN
INNER JOIN FINCORE_DB.FINCORE.ORGANIZATION_MASTER_CLUSTER_FINGATE_REPORT C ON C.ORGANIZATION_CLUSTER_ID  = A.ORGANIZATION_CLUSTER_ID
INNER JOIN EXTL_DB.CBDT.CBDT_NATURE_OF_BUSINESS_MASTER_DATA D ON B.NATURE_OF_BUSINESS = D.NOB_CODE
)T
)T)
T
WHERE T.ORGANIZATION_MASTER_ID = FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_MASTER_ID
'''





org_pincode_update = '''

UPDATE fincore_db.[Fincore_Analytics].[RISK_organization_SUMMARY_STG]
   SET PINCODE = B.pin
from
(select pan as P, pin
from
(
select  pan, try_cast(try_cast(pincode as int) as decimal(6,0)) as pin,  
rank() over(partition by pan order by legacy_batch_id desc) as orank
from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_EntityDetail]
where pan is not null and PinCode is not null
) T
where orank= 1
)
B
where pan = B.P'''


master_data_pop_org_q2 = '''update Fincore_Analytics.[RISK_ORGANIZATION_SUMMARY_STG]
SET PAN = b.pan ,
GSTIN = b.gstin ,
FULL_NAME = b.Full_Name,
UNIQUE_COMPANY_ID = b.Unique_Company_Id,
DATE_OF_INCORPORATION = b.Date_Of_Incorporation,
IEC = b.Iec
from
(
select Organization_Master_ID as entity_id ,
pan ,
gstin,
Unique_Company_Id,
Iec,
Full_Name,
Date_Of_Incorporation
from Fincore.Organization_Master
) b
where ORGANIZATION_MASTER_ID = b.entity_id'''


In [11]:
master_data_pop_per_income = '''update [FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]
set cbdt_income_range = b.income_range
from (
select pm.ORGANIZATION_Master_Id as entity_id,
case when itr.Gross_Total_Income_Range = 0 then null
when itr.Gross_Total_Income_Range = 1 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 2 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 3 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 4 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 5 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 6 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 7 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 8 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 9 then 10000000 end as Income_range 
from FINCORE_DB.FINCORE.ORGANIZATION_Master pm inner join EXTL_DB.CBDT.CBDT_Itr_Data itr
on itr.Pan =pm.pan inner join   EXTL_DB.CBDT.CBDT_Income_Range   cbdtir
on itr.Gross_Total_Income_Range=cbdtir.Income_Range 
group by pm.ORGANIZATION_Master_Id,cbdtir.Max_Value_Lakh,cbdtir.Min_Value_Lakh,Gross_Total_Income_Range
)b

where ORGANIZATION_master_id = b.entity_id'''


per_trx_update_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set TRAN_AMNT_CREDIT = round(average,2)
from (
select w.ORGANIZATION_Master_Id as ORGANIZATION_Master_Id,sum(w.monthlyaverage)as average
from 
(select PMAM.ORGANIZATION_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
avg(amount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
--sum(amount)/Datediff(Month,transactionDate,getdate())  as monthlyaverage
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo with (nolock)
where
accountNumber is not null and accountNumber !=''
and depositeWithdrawl ='deposit'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.ORGANIZATION_Master_Id 

union

select PMAM.ORGANIZATION_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TcOne with (nolock)
where 
accountNumber is not null and accountNumber !=''
and depositWithdrawal ='deposit'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.ORGANIZATION_Master_Id

union

select PMAM.ORGANIZATION_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select beneficiaryAcNo ,
AVG(Amount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsOne with (nolock)
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.ORGANIZATION_Master_Id

union

select PMAM.ORGANIZATION_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select beneficiaryAcNo ,
AVG (amount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsTwo with (nolock)
where 
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t
on t.beneficiaryAcNo=Am.Account_Number 
where Rank  = 1
group by PMAM.ORGANIZATION_Master_Id

union

select PMAM.ORGANIZATION_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select beneficiaryAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsThree with (nolock)
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t 
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.ORGANIZATION_Master_Id

union 

select PMAM.ORGANIZATION_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select beneficiaryAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_GenReporting with (nolock)
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
group by beneficiaryAcNo ,transactionDate)t 
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.ORGANIZATION_Master_Id
)w
group by w.ORGANIZATION_master_id
)X
where FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id = X.ORGANIZATION_Master_Id'''


per_trx_update_debit = '''
update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set TRAN_AMNT_DEBIT = round(average,2)
from (
select w.ORGANIZATION_Master_Id as ORGANIZATION_Master_Id,sum(w.monthlyaverage)as average
from 
(select PMAM.ORGANIZATION_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
avg(amount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
--sum(amount)/Datediff(Month,transactionDate,getdate())  as monthlyaverage
from  FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo with (nolock)
where
accountNumber is not null and accountNumber !=''
and depositeWithdrawl ='Withdrawal'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.ORGANIZATION_Master_Id

union

select PMAM.ORGANIZATION_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
from  FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TcOne with (nolock)
where 
accountNumber is not null and accountNumber !=''
and depositWithdrawal ='Withdrawal'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.ORGANIZATION_Master_Id

union

select PMAM.ORGANIZATION_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select SENDERACNO ,
AVG(Amount) as average,
RANK() OVER (PARTITION BY SENDERACNO ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsOne with (nolock)
where  
SENDERACNO is not null and SENDERACNO !=''
and transactionType ='DEBIT'
group by SENDERACNO ,transactionDate)t
on t.SENDERACNO=Am.Account_Number 
where Rank = 1
group by PMAM.ORGANIZATION_Master_Id
 
union

select PMAM.ORGANIZATION_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select senderAcNo ,
AVG (amount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsTwo with (nolock)
where 
senderAcNo is not null and senderAcNo !=''
and transactionType ='DEBIT'
group by senderAcNo ,transactionDate)t
on t.senderAcNo=Am.Account_Number 
where Rank  = 1
group by PMAM.ORGANIZATION_Master_Id
 
union
 
select PMAM.ORGANIZATION_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select senderAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsThree with (nolock)
where  
senderAcNo is not null and senderAcNo !=''
and transactionType ='DEBIT'
group by senderAcNo ,transactionDate)t 
on t.senderAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.ORGANIZATION_Master_Id

union 

select PMAM.ORGANIZATION_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.ORGANIZATION_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select senderAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_GenReporting with (nolock)
where  
senderAcNo is not null and senderAcNo !=''
group by senderAcNo ,transactionDate)t 
on t.senderAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.ORGANIZATION_Master_Id
)w
group by w.ORGANIZATION_master_id
)X
where 
FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id = X.ORGANIZATION_Master_Id'''


peer_grouping_columns = '''update fincore_db.fincore_analytics.risk_organization_summary_stg set peer_group_amnt = case
when (cbdt_income_range is not null and cbdt_income_range!=0) then cbdt_income_range
when (tran_amnt_credit is not null and tran_amnt_credit!=0) then tran_amnt_credit 
when (tran_amnt_debit is not null and tran_amnt_debit!=0) then tran_amnt_debit
else 0
end
'''

In [12]:
# ------Mobile Count 
MOBILE_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set MOBILE_COUNT= k.no_of_contacts from 
(
SELECT  distinct  PMCM.ORGANIZATION_Master_Id,  
      Count(Distinct PMCM.Contact_Master_Id) as no_of_contacts    
      FROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Contact_Master PMCM  with(nolock)
      INNER JOIN [FINCORE_DB].[Fincore].Contact_Master CM      with(nolock)
      ON PMCM.Contact_Master_Id = CM.Contact_Master_Id 
      WHERE PMCM.Is_Active_Flag = 1 AND CM.Contact_Type = 'M'  GROUP BY PMCM.ORGANIZATION_Master_Id)k
      where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''





# ------TELEPHONE Count 
TELEPHONE_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set TELEPHONE_COUNT= k.no_of_contacts from 
(
SELECT  distinct  PMCM.ORGANIZATION_Master_Id,  
      Count(Distinct PMCM.Contact_Master_Id) as no_of_contacts    
      FROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Contact_Master PMCM  with(nolock)
      INNER JOIN [FINCORE_DB].[Fincore].Contact_Master CM      with(nolock)
      ON PMCM.Contact_Master_Id = CM.Contact_Master_Id 
      WHERE PMCM.Is_Active_Flag = 1 AND CM.Contact_Type = 'T'  GROUP BY PMCM.ORGANIZATION_Master_Id)k
      where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

# ------Email Count 
EMAIL_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set EMAIL_COUNT= k.no_of_emailid from 
(
SELECT B.ORGANIZATION_MASTER_ID,  
  COUNT(DISTINCT A.EMAIL_ID) as no_of_emailid       
  FROM  [FINCORE_DB].[Fincore].ORGANIZATION_CLUSTER A     with(nolock)
  INNER JOIN  [FINCORE_DB].[Fincore].ORGANIZATION_MASTER_CLUSTER_FINGATE_REPORT B   with(nolock)
  ON A.ORGANIZATION_CLUSTER_ID = B.ORGANIZATION_CLUSTER_ID       
  WHERE A.EMAIL_ID IS NOT NULL    and A.status_active_flag = 1 and    B.is_active_flag =1  
  GROUP BY B.ORGANIZATION_Master_Id)k
      where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

# ------WATCHLIST COUNT 
WATCHLIST_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set WATCHLIST_COUNT= k.watchlist_count from 
(
select  pm.ORGANIZATION_Master_Id,     
               count(w.Agency_Name) as watchlist_count         
               from  FINCORE_DB.Fincore.ORGANIZATION_Master PM       with(nolock)   
               inner join [FINCORE_DB].[Fincore].[Watchlist] W      with(nolock)  
               on PM.ORGANIZATION_MASTER_ID = W.Entity_Master_Id            
               where w.Entity_Type = 'I'   and PM.Is_Active_Flag =  1   and W.Is_Active_Flag = 1 group by pm.ORGANIZATION_Master_Id )k
                 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''


# -----STR_REPORT_COUNT
STR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set STR_REPORT_COUNT = K.STR_COUNT from
(SELECT   PMCFR.ORGANIZATION_Master_Id,      
                       COUNT( 1 ) AS STR_COUNT      
                       FROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR'  GROUP BY PMCFR.ORGANIZATION_Master_Id )K
                       where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''


# -----CTR_REPORT_COUNT
CTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set CTR_REPORT_COUNT = K.CTR_COUNT from
(SELECT   PMCFR.ORGANIZATION_Master_Id,      
                       COUNT( 1 ) AS CTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR with (nolock) 
                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR'  GROUP BY PMCFR.ORGANIZATION_Master_Id )K
                       where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''


# -----NTR_REPORT_COUNT
NTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set NTR_REPORT_COUNT = K.NTR_COUNT from
(SELECT   PMCFR.ORGANIZATION_Master_Id,      
                       COUNT( 1 ) AS NTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR'  GROUP BY PMCFR.ORGANIZATION_Master_Id )K
                       where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

# -----PTR_REPORT_COUNT
PTR_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set PTR_REPORT_COUNT = K.PTR_COUNT from
(SELECT   PMCFR.ORGANIZATION_Master_Id,      
                       COUNT( 1 ) AS PTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'PTR'  GROUP BY PMCFR.ORGANIZATION_Master_Id )K
                       where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''


# ---------------CBWTR_REPORT_COUNT
CBWTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set CBWTR_REPORT_COUNT = K.CBWTR_COUNT from
(SELECT   PMCFR.ORGANIZATION_Master_Id,      
                       COUNT( 1 ) AS CBWTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CBWTR'  GROUP BY PMCFR.ORGANIZATION_Master_Id )K
                       where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

# -----CCR_REPORT_COUNT
CCR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set CCR_REPORT_COUNT = K.CCR_COUNT from
(SELECT   PMCFR.ORGANIZATION_Master_Id,      
                       COUNT( 1 ) AS CCR_COUNT      
                       FROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CCR'  GROUP BY PMCFR.ORGANIZATION_Master_Id )K
                       where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

# ----str_amount_debit
str_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set str_amount_debit = k.str_amount_debit from


(select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as str_amount_debit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
(( PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''



#      ----str_amount_credit

str_amount_credit =  '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set str_amount_credit = k.str_amount_credit from

 (select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as str_amount_credit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with (nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR' )and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

# ----ctr_amount_debit
ctr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set ctr_amount_debit = k.ctr_amount_debit from


(select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as ctr_amount_debit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

#  ----ctr_amount_credit

ctr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set ctr_amount_credit = k.ctr_amount_credit from

 (select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as ctr_amount_credit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

# ----ntr_amount_debit
ntr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set ntr_amount_debit = k.ntr_amount_debit from


(select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as ntr_amount_debit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR' )and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

#      ----ntr_amount_credit

ntr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set ntr_amount_credit = k.ntr_amount_credit from
 (select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as ntr_amount_credit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

# ----cbwtr_amount_debit
cbwtr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set cbwtr_amount_debit = k.cbwtr_amount_debit from


(select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as cbwtr_amount_debit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CBWTR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

#      ----cbwtr_amount_credit

cbwtr_amount_credit = ''' update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set cbwtr_amount_credit = k.cbwtr_amount_credit from

 (select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as cbwtr_amount_credit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CBWTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

# ----ccr_amount_debit
ccr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set ccr_amount_debit = k.ccr_amount_debit from


(select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as ccr_amount_debit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CCR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

#      ----ccr_amount_credit
ccr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG set ccr_amount_credit = k.ccr_amount_credit from
 (select PMCFR.ORGANIZATION_Master_Id,SUM(t.TRAN_AMNT) as ccr_amount_credit from [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CCR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.ORGANIZATION_Master_Id)k
 where k.ORGANIZATION_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_Master_Id'''

In [13]:
tran_amnt = '''UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG SET TRAN_AMNT_DEBIT = T.TRAN_DEBIT
FROM (
SELECT ENTITY_ID,ENTITY_TYPE,CR_DR_DEP_WITH,SUM(TRAN_AMNT) AS TRAN_DEBIT   
 FROM  FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG  
WHERE ENTITY_TYPE = 'O' 
GROUP BY ENTITY_ID,ENTITY_TYPE,CR_DR_DEP_WITH
HAVING CR_DR_DEP_WITH = 'DEBIT'
)T
WHERE ENTITY_ID = ORGANIZATION_MASTER_ID


UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG SET TRAN_AMNT_CREDIT = T.TRAN_CREDIT
FROM (
SELECT ENTITY_ID,ENTITY_TYPE,COALESCE(CR_DR_DEP_WITH,'CREDIT') AS CR_DR_DEP_WITH ,SUM(TRAN_AMNT) AS TRAN_CREDIT   
 FROM  FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG  
WHERE ENTITY_TYPE = 'O' 
GROUP BY ENTITY_ID,ENTITY_TYPE,COALESCE(CR_DR_DEP_WITH,'CREDIT')
HAVING COALESCE(CR_DR_DEP_WITH,'CREDIT') = 'CREDIT'
)T
WHERE ENTITY_ID = ORGANIZATION_MASTER_ID'''


In [14]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]


# Run these queries to populate risk_person_summary_stg table

In [15]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [16]:
#Run these queries to populate risk_person_summary_stg table

# Making a list of queries to populate the risk_person_summary_stg table
per_queries = [master_data_pop_org_q1,master_data_pop_org_q2 ,org_pincode_update ,org_occ_update ,master_data_pop_per_income,tran_amnt,peer_grouping_columns,MOBILE_COUNT,TELEPHONE_COUNT,EMAIL_COUNT,WATCHLIST_COUNT,STR_REPORT_COUNT,CTR_REPORT_COUNT,NTR_REPORT_COUNT,PTR_COUNT,CBWTR_REPORT_COUNT,CCR_REPORT_COUNT,str_amount_debit,str_amount_credit,ctr_amount_debit,ctr_amount_credit,ntr_amount_debit,ntr_amount_credit,cbwtr_amount_debit,cbwtr_amount_credit,ccr_amount_debit,ccr_amount_credit]
for query in per_queries:
    cursor.execute(query)
    print(namestr(query,globals())[0]," DONE!")
    conn.commit()
    
cursor.execute('''Delete from [FINCORE_DB].[Fincore_Analytics].[RISK_Organization_Incremental]''')   
conn.commit()
cursor.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_Organization_Incremental] 
(
Organization_MASTER_ID
)
SELECT Organization_MASTER_ID
FROM [FINCORE_DB].[Fincore_Analytics].[risk_Organization_summary_stg] with (nolock)''')
conn.commit()
conn.close()

master_data_pop_org_q1  DONE!
master_data_pop_org_q2  DONE!
org_pincode_update  DONE!
org_occ_update  DONE!
master_data_pop_per_income  DONE!
per_trx_update_credit  DONE!
per_trx_update_debit  DONE!
peer_grouping_columns  DONE!
MOBILE_COUNT  DONE!
TELEPHONE_COUNT  DONE!
EMAIL_COUNT  DONE!
WATCHLIST_COUNT  DONE!
STR_REPORT_COUNT  DONE!
CTR_REPORT_COUNT  DONE!
NTR_REPORT_COUNT  DONE!
PTR_COUNT  DONE!
CBWTR_REPORT_COUNT  DONE!
CCR_REPORT_COUNT  DONE!
str_amount_debit  DONE!
str_amount_credit  DONE!
ctr_amount_debit  DONE!
ctr_amount_credit  DONE!
ntr_amount_debit  DONE!
ntr_amount_credit  DONE!
cbwtr_amount_debit  DONE!
cbwtr_amount_credit  DONE!
ccr_amount_debit  DONE!
ccr_amount_credit  DONE!


# Populate the risk_person_summary_stg_rules

In [18]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [19]:
#Populate the risk_person_summary_stg_rules


for query in rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) & (rules_df['PERSON_TYPE'] == 'Organization')]['Query'].to_list():
    pprint(query)
    cursor.execute(query)
    print("Done")
#     print(namestr(query,globals())[0]," DONE!")
    conn.commit()
    
conn.close()

('\r\n'
 '                  MERGE INTO '
 '[FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES]     \r\n'
 '\t\t\t\t  as target USING     \r\n'
 '\t\t\t\t  (            \r\n'
 'select cte.Organization_master_id,avg(cte.Income_Per_Year_Inr) as '
 'Income_Per_Year_Inr ,avg(cte.cc_Percentage_grtr_thn_income) \r\n'
 '\t\t\t\t  as cc_Percentage_grtr_thn_income from(\r\n'
 ' select  distinct  pmcfr.ORGANIZATION_Master_Id, \r\n'
 '\t\t\t\t  cbdtir.Income_Per_Year_Inr,        \r\n'
 '\t\t\t\t  case when cbdtir.Income_Per_Year_Inr is not null       \r\n'
 '\t\t\t\t  then sum((fgtc1.transactionAmount - cbdtir.Income_Per_Year_Inr)/ '
 'cbdtir.Income_Per_Year_Inr)*100 end as    \r\n'
 '\t\t\t\t  cc_Percentage_grtr_thn_income     \r\n'
 '\t\t\t\t  from       \r\n'
 '\t\t\t\t  FINCORE_BRIDGE_DB.FINCORE_BRIDGE.fingate_entityDetail fpd  with '
 '(nolock)     \r\n'
 '\t\t\t\t  inner join fincore_db.Fincore.ORGANIZATION_Cluster pc with '
 '(nolock)  on fpd.reportId=pc.Report_Id and '
 

('\r\n'
 '                          MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES  \r\n'
 '\t\t\t\t\t\t  WITH (HOLDLOCK) as target USING (      \r\n'
 '\t\t\t\t\t\t  SELECT   PMCFR.ORGANIZATION_Master_Id,     \r\n'
 '\t\t\t\t\t\t  SUM( 1 ) AS STR_COUNT          \r\n'
 '\t\t\t\t\t\t  FROM '
 '[FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report PMCFR with '
 '(nolock) inner join \r\n'
 '\t\t\t\t\t\t  FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL k '
 'with(nolock)\r\n'
 '\t\t\t\t\t\t  on pmcfr.organization_master_id=k.organization_master_id\r\n'
 '\t\t\t\t\t\t  INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with '
 '(nolock) \r\n'
 '\t\t\t\t\t\t  ON PMCFR.Report_Type_Id = B.id_     \r\n'
 '\t\t\t\t\t\t  inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT rr with '
 '(nolock) \r\n'
 '\t\t\t\t\t\t  on PMCFR.Batch_Id=rr.BATCH_ID and '
 'PMCFR.Report_Id=rr.REPORT_ID  \r\n'
 "\t\t\t\t\t\t  WHERE PMCFR.Is_Active_Flag = 1 and 

Done
('\r\n'
 '            MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES as    \r\n'
 '\t\t\ttarget USING (     \r\n'
 '\t\t\tselect temp.ORGANIZATION_Master_Id ,\r\n'
 '\t\t\tcase when temp.Income_Per_Year_Inr is not null  \r\n'
 '\t\t\tthen '
 '((sum(temp.mutual_investments)-temp.Income_Per_Year_Inr)/temp.Income_Per_Year_Inr)*100 '
 'end as  Mutual_Investment_percentage   \r\n'
 '\t\t\tfrom  (         \r\n'
 '\t\t\tselect pm.ORGANIZATION_Master_Id ,   \r\n'
 '\t\t\t(cbdtir.min_value_lakh + cbdtir.max_value_lakh)/2 as '
 'income_per_year_inr,     \r\n'
 '\t\t\tsum(fgts5.amount) as mutual_investments from '
 'fincore_db.Fincore.ORGANIZATION_Master pm with(nolock)  \r\n'
 '\t\t\tinner join '
 'Fincore_db.Fincore.ORGANIZATION_Master_Cluster_Fingate_Report pmcfr '
 'with(nolock)       \r\n'
 '\t\t\ton pm.ORGANIZATION_Master_Id=pmcfr.ORGANIZATION_Master_Id inner '
 'join \r\n'
 '\t\t\tFINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL k '
 'with(no

Done
('\r\n'
 '            MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_Summary_stg_RULES  \r\n'
 '\t\t\tWITH (HOLDLOCK) as target  USING (               \r\n'
 '\t\t\tSELECT   PMCFR.ORGANIZATION_Master_Id,     \r\n'
 '\t\t\tSUM( 1 ) AS NTR_COUNT            \r\n'
 '\t\t\tFROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report '
 'PMCFR  with (nolock) inner join \r\n'
 '\t\t\tFINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL k '
 'with(nolock) \r\n'
 '\t\t\ton PMCFR.ORGANIZATION_Master_Id=k.organization_master_id\r\n'
 '\t\t\tINNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  '
 'ON PMCFR.Report_Type_Id = B.id_     \r\n'
 '\t\t\tinner join FINCORE_DB.Fincore_Analytics.RISK_REPORT rr with (nolock)  '
 'on PMCFR.Batch_Id=rr.BATCH_ID and PMCFR.Report_Id=rr.REPORT_ID   \r\n'
 "\t\t\tWHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR' and "
 'rr.RISK_SCORE<5 GROUP BY PMCFR.ORGANIZATION_Master_Id      )  \r\n'
 '\t\t\tAS source   

Done
('\r\n'
 '                                          MERGE INTO '
 '[FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES]    \r\n'
 '\t\t\t\t\t\t\t\t\t\t  as target USING (       \r\n'
 '\t\t\t\t\t\t\t\t\t\t  SELECT B.ORGANIZATION_MASTER_ID,       \r\n'
 '\t\t\t\t\t\t\t\t\t\t  COUNT(DISTINCT A.EMAIL_ID) as no_of_Email_Ids    \r\n'
 '\t\t\t\t\t\t\t\t\t\t  FROM  [FINCORE_DB].[Fincore].ORGANIZATION_CLUSTER A   '
 'with (nolock)    \r\n'
 '\t\t\t\t\t\t\t\t\t\t  INNER JOIN  '
 '[FINCORE_DB].[Fincore].ORGANIZATION_MASTER_CLUSTER_FINGATE_REPORT B with '
 '(nolock)         \r\n'
 '\t\t\t\t\t\t\t\t\t\t  ON A.ORGANIZATION_CLUSTER_ID = '
 'B.ORGANIZATION_CLUSTER_ID inner join \r\n'
 '\t\t\t\t\t\t\t\t\t\t  '
 'FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL k '
 'with(nolock) \r\n'
 '\t\t\t\t\t\t\t\t\t\t  on '
 'B.ORGANIZATION_MASTER_ID=k.organization_master_id\r\n'
 '\t\t\t\t\t\t\t\t\t\t  \r\n'
 '\t\t\t\t\t\t\t\t\t\t  \r\n'
 '\t\t\t\t\t\t\t\t\t\t  WHERE A.EMAIL_ID 

Done
('\r\n'
 '                        MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES   \r\n'
 '\t\t\t\t\t\tWITH (HOLDLOCK) as target USING        \r\n'
 '\t\t\t\t\t\t(          \r\n'
 '\t\t\t\t\t\tselect  PMAM.ORGANIZATION_Master_Id ,        \r\n'
 '\t\t\t\t\t\tCount(Distinct PMAM.Account_Master_Id) as No_Of_Accounts,   \r\n'
 "\t\t\t\t\t\tCount(Distinct ( cast (am.Re_Id as varchar) +'|'+ am.Branch_code "
 ') ) as No_Of_Banks     \r\n'
 '\t\t\t\t\t\tFROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Account_Master '
 'PMAM with (nolock)   inner join \r\n'
 '\t\t\t\t\t\tFINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL k '
 'with(nolock) \r\n'
 '\t\t\t\t\t\ton PMAM.ORGANIZATION_Master_Id =k.organization_master_id\r\n'
 '\t\t\t\t\t\tINNER JOIN [FINCORE_DB].[Fincore].Account_Master AM with '
 '(nolock)         \r\n'
 '\t\t\t\t\t\tON PMAM.Account_Master_Id = AM.Account_Master_Id          \r\n'
 '\t\t\t\t\t\tWHERE PMAM.Is_Active_Flag = 1    and AM.Acc

In [1]:
print('Master_data_pop_all_data_ORG Completed.')

Master_data_pop_all_data_ORG Completed.
